#### 1. Establish a connection between Python and the Sakila database.

In [1]:
import pandas as pd # all time remember to import pandas library

psw = '.Aa654321.'

import mysql.connector

cnx = mysql.connector.connect(user = 'root',
                              password = psw,
                              host = 'localhost',
                              database = 'Sakila',
                              auth_plugin = 'mysql_native_password')

cursor = cnx.cursor()

cursor

#### 2.Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

- engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
- month: an integer representing the month for which rental data is to be retrieved.
- year: an integer representing the year for which rental data is to be retrieved.

The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [2]:
def rentals_month(engine,month,year):
    
    # first of all we define the query we will do
    query = f"""
        SELECT 
            *
        FROM
            rental
        WHERE
            MONTH(rental_date) = {month}
                AND YEAR(rental_date) = {year}
        """ 
    return pd.read_sql(query, engine) # return a DataFrame with the query selected

#### 3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [31]:
def rental_count_month(month: int, year: int) -> dict:
    """_summary_

    Args:
        month (int): _mes a analizar_
        year (int): _año a analizar_

    Returns:
        _type_: _DataFrame con los datos_
    """    
    # crea una nueva dataframe
    data_frame = rentals_month(cnx,month,year)
    
    # Agrupar por customer_id y contar el número de alquileres
    rental_counts = data_frame.groupby('customer_id').size().reset_index(name = 'rental_count')

    # Crear el nombre de la columna basado en el mes y el año
    head_name = f'rentals_{str(month).zfill(2).lstrip()[-2:]}_{str(year)}'  # OPCIONAL

    # # Renombrar la columna para el mes y año seleccionados
    rental_counts.rename(columns = {'rental_count': head_name}, inplace = True) # OPCIONAL

    # # Ordenar el DataFrame por la columna de alquileres
    rental_counts.sort_values(by = head_name, ascending = False, inplace = True)

    return rental_counts

#### 4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [29]:
def compare_rentals(month_1, year_1,month_2, year_2):
    
    # Creamos los dos dataframes para poder analizarlos llamando a la funcion rental_count_month
    df1 = rental_count_month(month_1, year_1)
    df2 = rental_count_month(month_2, year_2)
    
    # Merge the two DataFrames on 'customer_id'
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer')

    # Replace NaN values with 0
    combined_df.iloc[:, 1].fillna(0, inplace=True)
    combined_df.iloc[:, 2].fillna(0, inplace=True)

    # Calculate the difference between the two rental counts
    combined_df['difference'] = abs(combined_df.iloc[:, 1] - combined_df.iloc[:, 2])
    
    # Sort by customer_id in descending order
    combined_df.sort_values(by='customer_id', ascending=True, inplace=True)

    return combined_df

#### Testing

In [28]:
# inputs
# month_1 = int(input('Introduce un mes'))
# year_1 = int(input('Introduce un año'))
# month_2 = int(input('Introduce otro mes'))
# year_2 = int(input('Introduce otro año'))

from IPython.display import display
import pandas as pd

# Ejercicio 1
print(cnx.is_connected())

# Ejercicio 2
df_rentals_month = rentals_month(cnx, 5, 2005)
print("Tabla del ejercicio 2:")
display(df_rentals_month)

# Ejercicio 3
df_rental_count_month = rental_count_month(5, 2005)
print("Tabla del ejercicio 3:")
display(df_rental_count_month)

# Ejercicio 4
df_compare_rentals = compare_rentals(5, 2005, 6, 2005)
print("Tabla del ejercicio 4:")
display(df_compare_rentals)


True
Tabla del ejercicio 2:


C:\Users\Adriu\AppData\Local\Temp\ipykernel_12144\1091852485.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, engine) # return a DataFrame with the query selected


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


Tabla del ejercicio 3:


C:\Users\Adriu\AppData\Local\Temp\ipykernel_12144\1091852485.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, engine) # return a DataFrame with the query selected


,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


Tabla del ejercicio 4:


C:\Users\Adriu\AppData\Local\Temp\ipykernel_12144\1091852485.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, engine) # return a DataFrame with the query selected
C:\Users\Adriu\AppData\Local\Temp\ipykernel_12144\1091852485.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, engine) # return a DataFrame with the query selected


,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,2.0
520,4,0.0,6.0,6.0
3,5,3.0,5.0,2.0
...,...,...,...,...
516,595,1.0,2.0,1.0
517,596,6.0,2.0,4.0
518,597,2.0,3.0,1.0
597,598,0.0,1.0,1.0
